In [4]:
from codex import Codex

import logging
import json

from codex.database.espresso import run_helpdoc, generate_json
from codex.database.vasp import generate_json

logging.basicConfig(level=logging.WARNING)

In [ ]:
codex = Codex('example/vasp/INCAR_Fe3O4.vasp', 'vasp', '1686736265')

In [ ]:
from pymongo import MongoClient
from importlib import resources
import os
import re
from packaging import version
import datetime

client = MongoClient('localhost', 27017)
db_base_dir = resources.files("codex.database")

# Add to MongoDB

In [ ]:
base_db_dir = resources.files("codex.database")
json_files = os.listdir(os.path.join(base_db_dir, "json"))
espresso_versions = [
    match.group(1)
    for f in json_files
    for match in [re.match(r"espresso-([0-9]+\.[0-9]+(?:\.[0-9]+)?)", f)]
    if match
]
vasp_versions = [
    match.group(1) for f in json_files for match in [re.match(r"vasp-([0-9]+)", f)] if match
]

espresso_latest = max(espresso_versions, key=version.parse)
vasp_latest = str(max(map(int, vasp_versions)))
vasp_latest_date = datetime.datetime.fromtimestamp(int(vasp_latest))

In [ ]:
base_db_dir = resources.files("codex.database")
for v in espresso_versions:
    database_json = os.path.join(base_db_dir, "json", f"espresso-{v}.json")
    with open(database_json, "r") as f:
        database = json.load(f)

    # Now prepare to add to MongoDB
    this_mongo_db =  client['espresso-' + v.replace(".", "x")]
    for package_name, package in database.items():
        this_mongo_db[package_name].insert_many(package)

# VASP database

In [ ]:
database = generate_json()

In [ ]:
fudge_dates = True
# Change the date on 20 random entries in the database to 1980
import random
if fudge_dates and database:
    for i in range(30):
        title = random.choice(list(database['INCAR'].keys()))
        database['INCAR'][title]["last_revised"] = "1980-01-01T00:00:00Z"
        #print(f"Changed ['INCAR']['{title}'] to 1980-01-01T00:00:00Z")

In [ ]:
tag = "ICHARG"
text = database[tag]["text"]
sections = text.get_sections()
options = []
#for section in sections:
#    if section.title is not None:
#        # Strip title of whitespace (leading and trailing)
#        if section.title.strip() == "Tag options":
#            lists = section.get_lists()
wikilink="*{{TAG|"+tag+"}}"
lists = text.get_lists()
for lll in lists:
    for item in lll.fullitems:
        if item.startswith(wikilink):
            options.append(item.split(wikilink)[1].strip())

print(options)
#options_dict = {}
#for option in options:
#    setting = wtp.parse(option.split(":")[0]).plain_text()
#    explanation = wtp.parse(option.split(":")[1]).plain_text()
#    options_dict.update({setting: explanation})

#options_dict
tag = "Available PAW potentials"
text = database[tag]["text"]
sections = text.get_sections()[1:3]
bolds = []
for section in sections:
    print(section.title)
    for b in section.get_bolds():
        try:
            float(b.plain_text())
        except:
            print("Found recommended pseudo: " + b.plain_text())
            # TODO: doesn't work for the GW ones....
            bolds.append(b.plain_text())

#bolds

# QE database

In [5]:
versions = ['6.7', '6.8', '7.0', '7.1', '7.2']
database_dir = resources.files("codex.database")

for v in versions:
    #run_helpdoc(v)
    db = generate_json(v)

# Generate Codex

In [ ]:
from importlib import resources
filename = 'example/sr3pbo_bands.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
codex = Codex(filename, version)


In [ ]:
print(codex.filename)
print(codex.code)
print(codex.package)
print(codex.tags['control'])

In [ ]:
from importlib import resources
import os
from base64 import b64decode
import json
import shutil

dbversion='7.2'
dir='static'

base_db_dir = resources.files("codex.database")
code = "qe"  # Need better terminology
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)

working_dir = os.path.abspath(dir)
tags_dir = os.path.join(working_dir, "tags-qe")
if os.path.exists(tags_dir):
    shutil.rmtree(tags_dir, ignore_errors=True)
os.mkdir(tags_dir)
for p, package in database.items():
    print(p)
    os.mkdir(os.path.join(tags_dir, p))
    for nl, namelist in package.items():
        print(nl)
        os.mkdir(os.path.join(tags_dir, p, nl))
        for tag, v in namelist.items():
            print(tag, v)
            if v["html"] != 'No documentation was found for this tag.':
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(tags_dir, p, nl, f"{tag}.html")
                with open(path, "w") as f:
                    f.write(webpage)